In [0]:
# Download and unzip the complete dogs-vs-cats dataset

%env DIR=cats_and_dogs

! curl -o dogs-vs-cats.zip https://storage.googleapis.com/peddy-ai-dl-data/deeplearning-repo/dogs-vs-cats.zip

! unzip -o dogs-vs-cats.zip -d $DIR
! mkdir $DIR/original_data
! unzip -o $DIR/test1.zip -d $DIR/original_data
! unzip -o $DIR/train.zip -d $DIR/original_data

In [0]:
# Create train/validation/test datasets from original data
import os, shutil

USE_DATA_AUGMENTATION = True

base_dir = os.path.join(os.getcwd(), 'cats_and_dogs')
original_dataset_dir = os.path.join(base_dir, 'original_data', 'train')

data_segment_names = ['train', 'validation', 'test']
labels = ['cat', 'dog']

data_segments = {
	data_segment_names[0]: (0, 1000),
	data_segment_names[1]: (1000, 1500),
	data_segment_names[2]: (1500, 2000)
}

def mkdir(dir):
	try:
		os.mkdir(dir)
	except OSError:
		print('{} not created - alread exists'.format(dir), dir)

def prepare_data():
	# Make data segments directories
	for segment in data_segment_names:
		segment_dir = os.path.join(base_dir, segment)
		mkdir(segment_dir)
		# Make label directories for each data segment
		for label in labels:
			label_segment_dir = os.path.join(segment_dir, label + 's')
			mkdir(label_segment_dir)
			# Copy data over for each data segment-label pair
			fnames = [(label+'.{}.jpg').format(i) for i in range(data_segments[segment][0], data_segments[segment][1])]
			for fname in fnames:
				src = os.path.join(original_dataset_dir, fname)
				dst = os.path.join(label_segment_dir, fname)
				shutil.copyfile(src, dst)

prepare_data()

In [0]:
! pip install tf-nightly-2.0-preview

In [4]:
import tensorflow as tf

from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print(tf.__version__)

2.0.0-dev20190413


In [0]:
# Build a network with 4 convolution-maxpooling blocks
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=1e-4),
			  loss='binary_crossentropy',
			  metrics=['acc'])

In [7]:
# Create data generators for training and validation
train_dir = os.path.join(os.getcwd(), 'cats_and_dogs', 'train')
validation_dir = os.path.join(os.getcwd(), 'cats_and_dogs', 'validation')

train_datagen = ImageDataGenerator(rescale=1./255)
if (USE_DATA_AUGMENTATION):
  print("Using data augmentation")
  train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
	train_dir,
	target_size=(150, 150),
	batch_size=20,
	class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
	validation_dir,
	target_size=(150, 150),
	batch_size=20,
	class_mode='binary')

Using data augmentation
Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [10]:
# Train the model using training and validation generators
history = model.fit_generator(
	train_generator,
	steps_per_epoch=100,
	epochs=10,
	validation_data=validation_generator,
	validation_steps=1)

Epoch 1/10
100/100 [==============================] - 107s 1s/step - loss: 0.6868 - acc: 0.5490 - val_loss: 0.7005 - val_acc: 0.4500
Epoch 2/10
100/100 [==============================] - 107s 1s/step - loss: 0.6695 - acc: 0.5835 - val_loss: 0.6828 - val_acc: 0.5000
Epoch 3/10
100/100 [==============================] - 107s 1s/step - loss: 0.6523 - acc: 0.6045 - val_loss: 0.6846 - val_acc: 0.5000
Epoch 4/10
100/100 [==============================] - 107s 1s/step - loss: 0.6298 - acc: 0.6470 - val_loss: 0.5191 - val_acc: 0.7500
Epoch 5/10
100/100 [==============================] - 107s 1s/step - loss: 0.6185 - acc: 0.6580 - val_loss: 0.5219 - val_acc: 0.8000
Epoch 6/10
100/100 [==============================] - 109s 1s/step - loss: 0.5990 - acc: 0.6735 - val_loss: 0.5134 - val_acc: 0.8000
Epoch 7/10
100/100 [==============================] - 108s 1s/step - loss: 0.6019 - acc: 0.6830 - val_loss: 0.4439 - val_acc: 0.8500
Epoch 8/10
100/100 [==============================] - 107s 1s/step - 

In [11]:
test_dir = os.path.join(os.getcwd(), 'cats_and_dogs', 'test')
test_generator = test_datagen.flow_from_directory(
  test_dir,
  target_size=(150, 150),
  batch_size=20,
  class_mode='binary')

loss, accuracy = model.evaluate_generator(test_generator)
print(accuracy)

Found 1000 images belonging to 2 classes.
0.695
